# Computer Vision: Final Project

Submitted by: [ **Adi Goldraich and Rotem Hayut** ]


In [3]:
import numpy as np
import time
import imutils
import cv2

### Counting Items:

In our priject we created a system that counts all the people entering and leaving the front (and only) entrance of a building.
In this way in a case of emergency (fire for example) the evacuation forces will know how many people were in the building in order to know if there are people in danger.

In [4]:
avg = None
video = cv2.VideoCapture("entrance.mp4")
xvalues = list()
motion = list()
count1 = 24  #the number of people in the building we defined, starts the count of "In"
count2 = 0   #starts the count of "Out"

def find_majority(k):
    myMap = {}
    maximum = ( '', 0 ) #occurring element
    for n in k:
        if n in myMap: myMap[n] += 1
        else: myMap[n] = 1
        if myMap[n] > maximum[1]: maximum = (n,myMap[n])   # Keep track of maximum on the go
    return maximum

while 1:
    ret, frame = video.read()
    if ret==True:
        flag = True
        text=""

        frame = cv2.resize(frame,(640,360))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (21, 21), 0)

        if avg is None:
            avg = gray.copy().astype("float")
            continue

        cv2.accumulateWeighted(gray, avg, 0.5)
        frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg))
        thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        (_, cnts, _) = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        for c in cnts:
            if cv2.contourArea(c) < 5000:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            xvalues.append(x)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            flag = False
        no_x = len(xvalues)
    
        if (no_x > 2):
            difference = xvalues[no_x - 1] - xvalues[no_x - 2]
            if(difference > 0):
                motion.append(1)
            else:
                motion.append(0)

        if flag is True:
            if (no_x > 5):
                val, times = find_majority(motion)
                if val == 1 and times >= 15:
                    count1 += 1
                else:
                    count2 += 1
                
            xvalues = list()
            motion = list()
            
#adding text and drawing on the frame:
        cv2.line(frame, (470, 0), (440,480), (0,255,0), 2)
        cv2.putText(frame, "People in the building:", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        cv2.putText(frame, "In: {}".format(count1), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.putText(frame, "Out: {}".format(count2), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.putText(frame, "Total: {}".format(count1-count2), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        cv2.imshow("Security Camera",frame)

    
        if (cv2.waitKey(1) & 0xff == 27): # ESC key pressed?
            break
    else:
        break

    
video.release()
cv2.destroyAllWindows()

## Thank You!